In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sms-spam-collection-dataset/spam.csv


In [2]:
df = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv',encoding='latin1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
df = df[["v1","v2"]]

In [4]:
df.rename(columns={'v1': 'Spam or ham','v2':'Message'}, inplace=True)
df.head()

,Spam or ham,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
type(df)

pandas.core.frame.DataFrame

In [6]:
df.describe()

,Spam or ham,Message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Spam or ham  5572 non-null   object
 1   Message      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [8]:
df.drop_duplicates(inplace=True)

In [9]:
df['spam'] = df["Spam or ham"].apply(lambda x:1 if x=="spam" else 0)
df.head()

,Spam or ham,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.Message,df.spam,test_size=0.25)

In [11]:
y_train.head()

78      0
321     0
4659    0
1156    0
4531    0
Name: spam, dtype: int64

In [12]:
y_train.shape

(3876,)

In [13]:
y_test.head()

2255    0
5028    1
330     0
886     0
2892    0
Name: spam, dtype: int64

**converting text data into numerical format based on the frequency of words**

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

# Initializing CountVectorizer
v = CountVectorizer()

# Transforming the text data (X_train.values) into a matrix of token counts
X_train_count = v.fit_transform(X_train.values)

# Converting the matrix of token counts to a dense array representation (showing the first three rows)
X_train_count.toarray()[:3]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

**Build and Train Models using Naive Bayes classifiers**

In [15]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_count,y_train)

MultinomialNB()

**Evaluation of model performance**

In [16]:
emails = [
    "Hey Ashfa, How are you",
    'Upto 20% discount on parking,exclusive offer just for you'
]
emails_count = v.transform(emails)
model.predict(emails_count)

array([0, 1])

In [17]:
X_test_count = v.transform(X_test)
model.score(X_test_count,y_test)

0.982985305491106

In [18]:
from sklearn.pipeline import Pipeline
clf = Pipeline([
    ('vetorizer', CountVectorizer()),
    ('nb',MultinomialNB())
])

In [19]:
clf.fit(X_train,y_train)

Pipeline(steps=[('vetorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [20]:
clf.predict(emails)

array([0, 1])

**Implementation using scikit-learn with TF-IDF and a Naive Bayes classifier**

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report

# Convert text data to numerical features using TF-IDF
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Initialize and train Naive Bayes classifier
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = model.predict(X_test_tfidf)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")


Accuracy: 0.9528228924980665
Classification Report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      1133
           1       1.00      0.62      0.76       160

    accuracy                           0.95      1293
   macro avg       0.97      0.81      0.87      1293
weighted avg       0.96      0.95      0.95      1293

